In [ ]:
%matplotlib inline
import os
import shutil
import glob

import radiomics
from radiomics import featureextractor

import pandas as pd
import SimpleITK as sitk


# 1. Feature extraction

## 1.1 setup working directory

In [ ]:
work_dir = '/home/peng/01_data/02_radiomics/demo'
data_dir = os.path.join(work_dir,'data')
result_dir = os.path.join(work_dir,'result')


tmp_volume = os.path.join(work_dir,'TmpVolumeDir')
tmp_mask = os.path.join(work_dir,'TmpMaskDir')

In [ ]:
## remove files from temparal directory
if os.path.isdir(tmp_volume):
    shutil.rmtree(tmp_volume)

if os.path.isdir(tmp_mask):
    shutil.rmtree(tmp_mask)
# --------------------------------------------------------
# check if dirs exist
dir_list = [result_dir,tmp_volume,tmp_mask]
for dir in dir_list:
    if not os.path.exists(dir):
        os.makedirs(dir)

In [ ]:
# for users 
# specify image nii dir and mask nii dir

# Create lists to store the paths to the images and masks
image_paths = {}
mask_paths = {}

# Iterate over the patient directories in data_dir
for patient_dir in os.listdir(data_dir):
    patient_path = os.path.join(data_dir, patient_dir)
    
    # Look for 'image.nii.gz' in the patient directory and add its path to image_paths
    image_file = glob.glob(os.path.join(patient_path, 'image.nii.gz'))
    if image_file:
        image_paths[patient_dir] = image_file[0]
    
    # Look for 'mask.nii.gz' in the p        img = io.read_image_sitk(Path(image_path))
        mask = io.read_segmentation_sitk(Path(mask_path), label=label)atient directory and add its path to mask_paths
    mask_file = glob.glob(os.path.join(patient_path, 'segmentation.nii.gz'))
    if mask_file:
        mask_paths[patient_dir] = mask_file[0]

# Sort the lists
# image_paths.sort()
# mask_paths.sort()

# Print the paths to the images and masks
for patient in image_paths:
    print(f'Patient: {patient}')
    print('Image:', image_paths[patient])
    print('Mask:', mask_paths[patient])

# check if they are in the same length
#TODO check id and pre/post
# check_dataset(volume_list, mask_list)

In [ ]:
# Create lists to store the paths to the images and masks
image_paths = []
mask_paths = []

# Iterate over the patient directories in data_dir
for patient_dir in sorted(os.listdir(data_dir)):
    patient_path = os.path.join(data_dir, patient_dir)
    
    # Look for 'image.nii.gz' in the patient directory and add its path to image_paths
    image_file = glob.glob(os.path.join(patient_path, 'image.nii.gz'))
    if image_file:
        image_paths.append(image_file[0])
    
    # Look for 'mask.nii.gz' in the patient directory and add its path to mask_paths
    mask_file = glob.glob(os.path.join(patient_path, 'segmentation.nii.gz'))
    if mask_file:
        mask_paths.append(mask_file[0])

# Print the paths to the images and masks
for i in range(len(image_paths)):
    print(f'Image: {image_paths[i]}')
    print(f'Mask: {mask_paths[i]}')

In [ ]:
def check_dataset(list_volume, list_mask):
    if len(list_volume) != len(list_mask):
        raise ValueError('There exists a mismatch between two datasets.')
    
# check if they are in the same length
#TODO check id and pre/post
check_dataset(image_paths, mask_paths)

## 1.2 Tumor delineation check
using Dice similarity to calc mask reproducibility, should set a propper cut-off value

In [ ]:
# Dice similarity
import nibabel as nib
import numpy as np
from scipy.spatial.distance import dice

def calculate_dice_scipy(mask1_path, mask2_path):
    # Load the masks
    mask1 = nib.load(mask1_path).get_fdata()
    mask2 = nib.load(mask2_path).get_fdata()

    # Binarize the masks
    mask1 = np.where(mask1 > 0, 1, 0)
    mask2 = np.where(mask2 > 0, 1, 0)

    # Flatten the masks
    mask1 = mask1.flatten()
    mask2 = mask2.flatten()

    # Calculate Dice dissimilarity
    dice_dissimilarity = dice(mask1, mask2)

    # Calculate Dice similarity
    dice_similarity = 1. - dice_dissimilarity

    return dice_similarity

# Usage:
# dice = calculate_dice_scipy('path_to_mask1.nii.gz', 'path_to_mask2.nii.gz')
# print(dice)

## 1.3 Feature extraction

In [ ]:
# setup parameters
params = os.path.join('/home/peng/00_github/03_radiomics/240602_auto-sklearn_SP/examples/examples_jupyter/Radiomics/Radiomics_Settings', 'exampleMR_1mm_SV.yaml')

In [ ]:
 # Instantiate a pandas data frame to hold the results of all patients
featureVector = pd.DataFrame()
sub = pd.DataFrame()
# Iterate over the image and mask paths
for image_path, mask_path in zip(image_paths, mask_paths):
    # Extract the features
    # print(f'Extracting features for {image_path}...')
    # print(f'Extracting features for {mask_path}...')
    img = sitk.ReadImage(image_path)
    # img_array = sitk.GetArrayFromImage(img)
    mask = sitk.ReadImage(mask_path)
    # mask.SetSpacing(img.GetSpacing())
    # mask.SetOrigin(img.GetOrigin())
    # mask.SetDirection(img.GetDirection())
    mask.CopyInformation(img)
    # mask_array = sitk.GetArrayFromImage(mask)
    # Check if the image and mask match
    if img.GetSize() != mask.GetSize() or img.GetSpacing() != mask.GetSpacing() or img.GetOrigin() != mask.GetOrigin():
        print(f'Geometry mismatch between image {image_path} and mask {mask_path}')
    
    result = pd.Series(extractor.execute(img, mask))
    featureVector = featureVector._append(result, ignore_index=True)

    directories = os.path.normpath(image_path).split(os.sep)
    last_directories = directories[-2:-1]
    series = pd.Series(last_directories, index=['subject'])
    # Append the series to the DataFrame
    sub = sub._append(series, ignore_index=True)


In [ ]:
featureVector = featureVector.iloc[:, 22:]
featureVector.head()

In [ ]:
# readin the labels
label_df = pd.read_csv(os.path.join(data_dir, "labels.csv"))
label_df.head()

In [ ]:
sub_lable_feature = pd.concat([label_df, featureVector], axis=1)
sub_lable_feature.to_csv(os.path.join(result_dir, 'sub_lable_feature.csv'), index=False)



In [ ]:
sub_lable_feature = pd.read_csv(os.path.join(result_dir, 'sub_lable_feature.csv'))


In [ ]:
sub_lable_feature.dtypes

In [ ]:
featureVector = sub_lable_feature.iloc[:, 2:]

In [ ]:
featureVector.head()

# 2. Preprocessing 

## 2.1 intraclass correlation coefficients (ICCs) from different masks delineated by different doctor

In [ ]:
# simulate some random data to demonstrate the feature icc calculation

import numpy as np

featureVector_std_dev = featureVector.std()
featureVector.shape
random_df = pd.DataFrame(np.random.rand(*featureVector.shape), columns=featureVector.columns)* featureVector_std_dev
featureVector_2_df = featureVector + random_df

In [ ]:
import pandas as pd
import pingouin as pg

# Ensure both dataframes have the same columns in the same order
assert set(featureVector.columns) == set(featureVector_2_df.columns)
featureVector = featureVector[featureVector_2_df.columns]

# Initialize an empty dictionary to store the ICC values
icc_values = {}
# Loop over the columns
for column in featureVector.columns:
    # Create a new dataframe that combines the data from the same column in both dataframes
    df_tmp = featureVector[[column]].copy()
    df_tmp['rater'] = 'rater1'
    df_tmp['subject'] = df_tmp.index
    df_2 = featureVector_2_df[[column]].copy()
    df_2['rater'] = 'rater2'
    df_2['subject'] = df_2.index

    df = pd.concat([df_tmp, df_2])
    # df[column] = pd.to_numeric(df[column], errors='coerce')
    # # Calculate the ICC and store the result in the dictionary
    icc = pg.intraclass_corr(data=df, targets='subject', raters='rater', ratings=column,nan_policy="omit")['ICC'].values[2] # ICC ICC(3,1)
    icc_values[column] = icc

In [ ]:
icc_df = pd.DataFrame(list(icc_values.items()), columns=['Features', 'ICC'])
icc_selected = icc_df[icc_df['ICC'] > 0.965]
print(icc_selected.index)
print(icc_selected['Features'])

In [80]:
feature_icc_selected = featureVector[icc_selected['Features']]
feature_icc_selected.to_csv(os.path.join(result_dir, 'feature_icc_selected.csv'), index=False)
# feature_icc_selected['ID'] = merged_feature_df['ID']
# feature_icc_selected['diagnosis'] = merged_feature_df['diagnosis']

## 2.2 variance of each feature value 
low variance (<75%) will be removed

In [82]:
# calculate variance of each feature
# low variance (<75%) will be removed

from sklearn.feature_selection import VarianceThreshold

# Initialize a VarianceThreshold feature selector
selector = VarianceThreshold(threshold=0.1)  # Adjust the threshold as needed

# Fit the selector to the data and transform the data
feature_icc_var_selected = selector.fit_transform(feature_icc_selected)

# Convert back to DataFrame
feature_icc_var_selected = pd.DataFrame(feature_icc_var_selected, columns=feature_icc_selected.columns[selector.get_support()])

## 2.3 chi2 of each feature value 
best 10 feature based chi2 was selected

In [84]:
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2
# Find the minimum value in the DataFrame
min_value = feature_icc_var_selected.min().min()
# If the minimum value is less than 0, shift all values by its absolute value
if min_value < 0:
    feature_icc_var_selected += abs(min_value)

selector = SelectKBest(chi2, k=10)

feature_icc_var_chi2_selected = selector.fit_transform(feature_icc_var_selected, sub_lable_feature['diagnosis'])
feature_icc_var_chi2_selected.shape
feature_icc_var_chi2_selected = pd.DataFrame(feature_icc_var_chi2_selected, columns=feature_icc_var_selected.columns[selector.get_support()])
print(feature_icc_var_chi2_selected.columns)

Index(['original_shape_Maximum3DDiameter',
       'log-sigma-1-0-mm-3D_glszm_LargeAreaHighGrayLevelEmphasis',
       'log-sigma-1-0-mm-3D_glszm_SizeZoneNonUniformity',
       'log-sigma-3-0-mm-3D_glrlm_RunLengthNonUniformity',
       'log-sigma-5-0-mm-3D_firstorder_10Percentile',
       'log-sigma-5-0-mm-3D_firstorder_Maximum',
       'wavelet-HH_glrlm_ShortRunHighGrayLevelEmphasis',
       'wavelet-HH_glszm_SmallAreaHighGrayLevelEmphasis',
       'wavelet-HH_glszm_ZoneVariance', 'wavelet-LL_firstorder_Median'],
      dtype='object')


## 2.4 mutual information of each feature value 
best 10 feature based  mutual information was selected

In [86]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# Initialize a SelectKBest feature selector
selector = SelectKBest(mutual_info_classif, k=10)  # Adjust the number of features as needed

# Fit the selector to the data and transform the data
feature_selected = selector.fit_transform(feature_icc_var_selected, sub_lable_feature['diagnosis'])

# Convert back to DataFrame
feature_icc_var_mi_selected = pd.DataFrame(feature_selected, columns=feature_icc_var_selected.columns[selector.get_support()])

# Get the names of the selected features
selected_features = feature_icc_var_selected.columns[selector.get_support()]

print('Selected features:', selected_features)

Selected features: Index(['original_shape_Maximum3DDiameter',
       'log-sigma-1-0-mm-3D_firstorder_RobustMeanAbsoluteDeviation',
       'log-sigma-1-0-mm-3D_glszm_LargeAreaHighGrayLevelEmphasis',
       'log-sigma-1-0-mm-3D_glszm_SizeZoneNonUniformity',
       'log-sigma-3-0-mm-3D_glrlm_RunLengthNonUniformity',
       'log-sigma-5-0-mm-3D_firstorder_10Percentile',
       'wavelet-HH_firstorder_MeanAbsoluteDeviation',
       'wavelet-HH_glszm_GrayLevelVariance',
       'wavelet-HH_gldm_LargeDependenceLowGrayLevelEmphasis',
       'wavelet-LL_firstorder_Median'],
      dtype='object')


## 2.5 variance inflation factor (VIF)

If you want to select features that are least correlated with each other, you can use the concept of variance inflation factor (VIF). VIF measures the correlation and multicollinearity of features. A high VIF value for a feature means that feature is highly correlated with other features.

In [87]:
# calculate VIF between features
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for each feature
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(feature_icc_var_selected.values, i) for i in range(feature_icc_var_selected.shape[1])]
vif["features"] = feature_icc_var_selected.columns

# Sort by VIF Factor in ascending order
vif = vif.sort_values("VIF Factor")

# Select the features with the lowest VIF
selected_features = vif["features"].head(10)  # Adjust the number of features as needed

print('Selected features:', selected_features)

Selected features: 4     log-sigma-1-0-mm-3D_glszm_LargeAreaHighGrayLev...
15                        wavelet-HH_glszm_ZoneVariance
5       log-sigma-1-0-mm-3D_glszm_SizeZoneNonUniformity
6      log-sigma-3-0-mm-3D_glrlm_RunLengthNonUniformity
17                         wavelet-LL_firstorder_Median
0                      original_shape_Maximum3DDiameter
8                log-sigma-5-0-mm-3D_firstorder_Maximum
7           log-sigma-5-0-mm-3D_firstorder_10Percentile
12       wavelet-HH_glrlm_ShortRunHighGrayLevelEmphasis
14      wavelet-HH_glszm_SmallAreaHighGrayLevelEmphasis
Name: features, dtype: object


## 2.6 Pearson correlation coefficient
large correlation coefficient (>0.75) will be removed

## 2.7 t-test or Mann-Whitney U test

In [ ]:
# t-test between groups or Mann-Whitney U test
from scipy.stats import mannwhitneyu,ttest_ind,shapiro

# Define an empty list to hold the selected features
selected_features = []

# Perform the Mann-Whitney U test for each feature
for feature in feature_icc_var_selected.columns:
    group1 = feature_icc_var_selected.loc[sub_lable_feature['diagnosis'] == 0, feature]
    group2 = feature_icc_var_selected.loc[sub_lable_feature['diagnosis'] == 1, feature]
    
    # Test the normality of the feature in each group
    _, p1 = shapiro(group1)
    _, p2 = shapiro(group2)
    
    # If both groups are normally distributed, perform the t-test
    if p1 > 0.05 and p2 > 0.05:
        stat, p = ttest_ind(group1, group2)
    # Otherwise, perform the Mann-Whitney U test
    else:
        stat, p = mannwhitneyu(group1, group2)
    
    # If the p-value is less than 0.05, add the feature to the list of selected features
    if p < 0.05:
        selected_features.append(feature)

print('Selected features:', selected_features)


# Classification

The following example shows how to fit a simple classification model with
*auto-sklearn*.


In [ ]:
from pprint import pprint

import sklearn.datasets
import sklearn.metrics
from sklearn import model_selection
import autosklearn.classification

## Data Loading



In [ ]:
X, y = sklearn.datasets.load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y, random_state=1
)

## Build and fit a classifier



In [ ]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    memory_limit=16384,
    tmp_folder="/tmp/autosklearn_classification_example_tmp",
)
automl.fit(X_train, y_train, dataset_name="breast_cancer")

## View the models found by auto-sklearn



In [ ]:
print(automl.leaderboard())

## Print the final ensemble constructed by auto-sklearn



In [ ]:
pprint(automl.show_models(), indent=4)

## Get the Score of the final ensemble



In [ ]:
predictions = automl.predict(X_test)
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))